# This Utility demonstrates IDCS API's usage with respect to groups and app roles

### Pre-requisties -
Create a trusted app in IDCS with **Identity Domain Administrator** API access and **Client Credentials** grant. Gather it's `client ID` and `secret`. This is needed to call the REST API operations.

In [1]:
import requests, json, base64, urllib.parse

#### Enter Environment Details

In [ ]:
tenant = ""
clientId = ''
clientSecret = ''
 
domain = "identity.oraclecloud.com"
port = "443"
baseurl = 'https://' + tenant + '.' + domain + ':' + port
configurl =  baseurl + '/.well-known/idcs-configuration'

#### Get an Access Token for calling REST API's

In [ ]:
config = None
response = requests.get(configurl).text
config = json.loads(response)
tokenurl = config["openid-configuration"]["token_endpoint"]

grant_type = "client_credentials"
scope = "urn:opc:idm:__myscopes__"

basicauthHeader = base64.b64encode(bytes(clientId + ":" + clientSecret, 'utf-8')) 
reqdata = {'grant_type': grant_type, 'scope' : scope}
reqheaders = {'Authorization': 'Basic ' + basicauthHeader.decode("utf-8"), 'content-type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data = reqdata, headers = reqheaders)
accesstoken = response.json()["access_token"]
headers = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
print(accesstoken)

#### Define Reusuable Functions

In [ ]:
def createGroup(gname, accesstoken, baseurl):
    groupurl = baseurl + "/admin/v1/Groups"    
    filter = "?filter=displayName eq \"" + gname + "\""
    glist = requests.get(groupurl + filter, headers = headers).json()["Resources"]
    
    if len(glist) == 0:
        print("Creating group ", gname)
        
        grouppayload = {
          "displayName": gname,
          "urn:ietf:params:scim:schemas:oracle:idcs:extension:group:Group": {
            "creationMechanism": "api",
            "description": gname
            },
  
          "schemas": [
            "urn:ietf:params:scim:schemas:core:2.0:Group",
            "urn:ietf:params:scim:schemas:oracle:idcs:extension:group:Group",
            "urn:ietf:params:scim:schemas:extension:custom:2.0:Group"
          ]
        }
        response = requests.post(groupurl, data = json.dumps(grouppayload), headers = headers).json()
        print("Group {} created with ID {}".format(gname, response["id"]))
        return response["id"]
    else:
        print("Group {} already exists".format(gname))
        return glist[0]["id"]

def getGroupId(gname):
    groupurl = baseurl + "/admin/v1/Groups"    
    filter = "?filter=displayName eq \"" + gname + "\""
    glist = requests.get(groupurl + filter, headers = headers).json()["Resources"]
    
    if len(glist) > 0:
        return glist[0]["id"]
    else:
        return "DOESNOTEXIST"
    
def getAppId(aname):
    appurl = baseurl + "/admin/v1/Apps"
    
    filter = "?filter=displayName eq \"" + aname + "\""
    applist = requests.get(appurl + filter, headers = headers).json()["Resources"]
    if len(applist) > 0:
        return applist[0]["id"]
    else:
        return None
    
    
def grantAppToGroup(appid, gid):
    granturl = baseurl + "/admin/v1/Grants"
    
    grantpayload = {
      "app": {
            "value": appid
      },
      "grantMechanism": "ADMINISTRATOR_TO_GROUP",
      "grantee": {
        "value": gid,
        "type": "Group"
      },
      "schemas": [
        "urn:ietf:params:scim:schemas:oracle:idcs:Grant"
      ]
    }
        
    response = requests.post(granturl, data = json.dumps(grantpayload), headers = headers).json()
    
    if("app" in response.keys()):
        print(response["app"]["value"])
    else:
        print(response)
        
def grantAppRoleToGroup(approleid,appid,gid):
    granturl = baseurl + "/admin/v1/Grants"
    
    grantpayload = {
      "app": {
            "value": appid
      },
      "entitlement": {
        "attributeName": "appRoles",
        "attributeValue": approleid
      },
      "grantMechanism": "ADMINISTRATOR_TO_GROUP",
      "grantee": {
        "value": gid,
        "type": "Group"
      },
      "schemas": [
        "urn:ietf:params:scim:schemas:oracle:idcs:Grant"
      ]
    }
        
    response = requests.post(granturl, data = json.dumps(grantpayload), headers = headers).json()
    
    if("app" in response.keys()):
        print(response["app"]["value"])
    else:
        print(response)

def getAppDeatils(appid, filter=""):
    appurl = baseurl + "/admin/v1/Apps/" + appid + filter
    return requests.get(appurl, headers = headers).json()

 
def getapproleids(approlename):
    approleurl = baseurl + "/admin/v1/AppRoles"
    filter = "?filter=displayName eq \"" + approlename + "\""
    approlelist = requests.get(approleurl + filter, headers = headers).json()["Resources"]
    
    if(len(approlelist) > 0):
        return approlelist[0]["id"], approlelist[0]["app"]["value"]
    else:
        return None, None

def revokeAppRoleFromGroup(appolename, gname):
    gid = getGroupId(gname)
    approleid,appid = getapproleids(appolename)
    grantid="DOESNOTEXIST"
    
    granturl = baseurl + "/admin/v1/Grants"
    filter = "?filter=(grantMechanism eq \"ADMINISTRATOR_TO_GROUP\")"
    res = requests.get(granturl + filter, headers = headers)
    grantlist = requests.get(granturl + filter, headers = headers).json()["Resources"]
    for grant in grantlist:
        e = grant.get("entitlement", dict()).get("attributeValue","")
        g = grant.get("grantee", "")["value"]
        a = grant.get("app", "")["value"]
        if((a==appid) and (g==gid) and (e==approleid)):
            grantid = grant["id"]
            break
    res = requests.delete(granturl + "/" + grantid, headers = headers)
    print(res.text)
    
def revokeAppFromGroup(appname, gname):
    appid = getAppId(appname)
    gid = getGroupId(gname)
    grantid="DOESNOTEXIST"
    
    granturl = baseurl + "/admin/v1/Grants"
    filter = "?filter=(grantMechanism eq \"ADMINISTRATOR_TO_GROUP\")"
    res = requests.get(granturl + filter, headers = headers)
    grantlist = requests.get(granturl + filter, headers = headers).json()["Resources"]
    for grant in grantlist:
        g = grant.get("grantee", "")["value"]
        a = grant.get("app", "")["value"]
        if((a==appid) and (g==gid)):
            grantid = grant["id"]
            break
            
    res = requests.delete(granturl + "/" + grantid, headers = headers)
    print(res.text)
    

def deleteGroup(gname):
    gid = getGroupId(gname)
    groupurl = baseurl + "/admin/v1/Groups/" + gid
    
    res = requests.delete(groupurl, headers = headers)
    print(res.text)

**Create 2 groups**
*   < poc-number >-admin
*   < poc-number >-user


In [ ]:
pocnumber = "100"

In [ ]:
admingrp = pocnumber + "-admin"
usergrp = pocnumber + "-user"

admingrpid = createGroup(admingrp, accesstoken, baseurl)
usergrpid = createGroup(usergrp, accesstoken, baseurl)

print(admingrpid)
print(usergrpid)

**Assign BMCS-SAML Application to those groups**

In [ ]:
appname = "BMCS-SAML"

In [ ]:
appid = getAppId(appname)

if appid:
    grantAppToGroup(appid,admingrpid)
    grantAppToGroup(appid,usergrpid)
else:
    print("No Application found with the name {}".format(appname))

 **Lookup Security ID and Client ID for “OCI-Federation” trusted application (optional)**

In [ ]:
trustedappname = "OCI-Federation"

x = getAppDeatils(getAppId(trustedappname), filter="")
print("Security ID = {}".format(x['clientSecret']))
print("Client ID = {}".format(x['name']))

**Assign to a given set of Application Roles above groups**

e.g.
* assign DBaaS Admin role to <poc-number>-admin
* assign JaaS Admin role to <poc-number>-admin
* assign Compute.Compute_Monitor role to <poc-number>-user

In [ ]:
appolename = 'Compute.Compute_Monitor'

approleid,appid = getapproleids(appolename)
if(approleid):
    grantAppRoleToGroup(approleid,appid,usergrpid)
else:
     print("AppRole {} does not exist".format(appolename))

appolename = 'JaaS_Administrator'

approleid,appid = getapproleids(appolename)
if(approleid):
     grantAppRoleToGroup(approleid,appid,admingrpid)
else:
     print("AppRole {} does not exist".format(appolename))

**Rollback Changes done**



In [ ]:
# Revoke App Roles
revokeAppRoleFromGroup('Compute.Compute_Monitor', usergrp)
revokeAppRoleFromGroup('JaaS_Administrator', admingrp)

# Revoke App
revokeAppFromGroup(appname, usergrp)
revokeAppFromGroup(appname, admingrp)

# Delete Groups
deleteGroup(usergrp)
deleteGroup(admingrp)